In [1]:
import re, json

from typing import Union

import torch as tt

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from transformers import EvalPrediction
from datasets import Dataset

In [2]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")
tokenizer.pad_token = tokenizer.eos_token

/usr/lib/python3/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
with open("title_dataset_pretty_filtered.json", 'r', encoding="utf8") as inp:
    title_dataset = json.load(inp)

In [4]:
title_dataset_train, title_dataset_val, title_dataset_test = title_dataset["train"], title_dataset["val"], title_dataset["test"]
title_dataset_train = Dataset.from_list(title_dataset_train)
title_dataset_val = Dataset.from_list(title_dataset_val)
title_dataset_test = Dataset.from_list(title_dataset_test)

In [5]:
len(title_dataset_train), len(title_dataset_val), len(title_dataset_test)

(4375, 219, 242)

In [6]:
set(item["answer"] for item in title_dataset_train)

{'A', 'B', 'C', 'D'}

In [7]:
set(item["answer"] for item in title_dataset_val)

{'A', 'B', 'C', 'D'}

In [8]:
set(item["answer"] for item in title_dataset_test)

{'A', 'B', 'C', 'D'}

In [9]:
option_id_dict = {
    'A': 0, 'B': 1, 'C': 2, 'D': 3
}

def to_new_format(example: dict[str, Union[str, list[str]]]) -> str:
  example["options_ru"] = [option for option in example["options_ru"] if option]
  right_answer = example['options_ru'][option_id_dict[example['answer']]]
  #print(right_answer)
  outp = example['article_ru'] + "\n" + "ВОПРОС: Какое название лучше всего подойдёт для этого текста? "
  outp += f"ПРАВИЛЬНЫЙ ОТВЕТ: {right_answer}"
  outp += "\nНЕПРАВИЛЬНЫЕ ВАРИАНТЫ ОТВЕТА:"
  for option in example["options_ru"]:
      if option != right_answer:
          #print(option)
          outp += f"\n  {option}"
  #print(outp)
  #raise Exception
  return {"inp": outp}

In [10]:
title_dataset_train = title_dataset_train.map(to_new_format)
title_dataset_val = title_dataset_val.map(to_new_format)
title_dataset_test = title_dataset_test.map(to_new_format)

Map:   0%|          | 0/4375 [00:00<?, ? examples/s]

Map:   0%|          | 0/219 [00:00<?, ? examples/s]

Map:   0%|          | 0/242 [00:00<?, ? examples/s]

In [11]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["inp"]
    )
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

In [12]:
title_dataset_train = title_dataset_train.map(preprocess_function, batched=True)
title_dataset_val = title_dataset_val.map(preprocess_function, batched=True)
title_dataset_test = title_dataset_test.map(preprocess_function, batched=True)

Map:   0%|          | 0/4375 [00:00<?, ? examples/s]

Map:   0%|          | 0/219 [00:00<?, ? examples/s]

Map:   0%|          | 0/242 [00:00<?, ? examples/s]

In [14]:
print(title_dataset_test[220]["inp"])

Привет, дорогие мальчики и девочки! Ты знаешь, как быть здоровым ребенком? Вот некоторые правила, которым ты должен следовать.
Во - первых, есть разные продукты, особенно фрукты и овощи. У вас может быть любимая еда, но вам лучше есть что-то другое, если вы едите разные продукты, вы, вероятно, получите больше питательных веществ, в которых нуждается ваше тело.
Во-вторых, пить воду и молоко как можно чаще. Когда ты действительно хочешь пить, холодная вода - это "Нет". 1 выбор. Молоко - это отличный напиток, который может дать вам больше кальция, чтобы вырастить крепкие кости.
В-третьих, слушай свое тело. Как ты себя чувствуешь, когда наелся? Когда вы едите, обращайте внимание на то, как чувствует себя ваше тело и когда ваш желудок чувствует себя комфортно насыщенным. Слишком много еды не сделает тебя комфортным и толстым.
В-четвертых, ограничить время экрана. Время скриншота - это время, когда вы смотрите телевизор, DVD и видео или используете компьютеры. Приятно делать больше упражнени

In [13]:
#max([len(example["labels"]) for example in title_dataset_train]), max([len(example["input_ids"]) for example in title_dataset_train])

In [14]:
#max([len(example["labels"]) for example in title_dataset_test])

In [15]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [16]:
NUM_TRAIN_EPOCHS=20
BATCH_SIZE=1
#STEPS=1000

training_args = TrainingArguments(
    output_dir="./RuGPT3-RuRACE",
    evaluation_strategy="epoch",
    weight_decay=0.01,
    learning_rate=5e-5,
    load_best_model_at_end=True,   
    save_strategy="epoch",
    num_train_epochs=NUM_TRAIN_EPOCHS,
    save_total_limit=3,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    prediction_loss_only=True,
    gradient_checkpointing=True,
    logging_dir="./rugpt3-rurace-title-logs"
)

In [17]:
trainer = Trainer(
    model,
    args=training_args,
    train_dataset=title_dataset_train,
    eval_dataset=title_dataset_val,
    data_collator=data_collator
)

In [18]:
trainer.train()
trainer.save_model("Ru-RACE-title-best")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss
1,2.904200,2.797599
2,2.636300,2.780272
3,2.352900,2.797400
4,2.106400,2.847580
5,1.866600,2.928264
6,1.728600,3.060783
7,1.527000,3.120582
8,1.333600,3.169202
9,1.251700,3.235275
10,1.112400,3.301702


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


In [31]:
def model_predict(text: str, max_length: int=1000) -> str:
    input_ = tokenizer([text], return_tensors="pt")
    output = model.generate(
        input_["input_ids"].to(tt.device("cuda:0")),
        max_length=max_length
    )
    return tokenizer.batch_decode(output)[0]

In [32]:
def preprocess_dataset_text(text: str) -> str:
    find_str = "НЕПРАВИЛЬНЫЕ ВАРИАНТЫ ОТВЕТА:\n"
    split_id = text.find(find_str)
    split_id += len(find_str)
    return text[:split_id], text[split_id:]

In [38]:
inp, label = preprocess_dataset_text(title_dataset_test[220]["inp"])

In [39]:
print(inp)

Привет, дорогие мальчики и девочки! Ты знаешь, как быть здоровым ребенком? Вот некоторые правила, которым ты должен следовать.
Во - первых, есть разные продукты, особенно фрукты и овощи. У вас может быть любимая еда, но вам лучше есть что-то другое, если вы едите разные продукты, вы, вероятно, получите больше питательных веществ, в которых нуждается ваше тело.
Во-вторых, пить воду и молоко как можно чаще. Когда ты действительно хочешь пить, холодная вода - это "Нет". 1 выбор. Молоко - это отличный напиток, который может дать вам больше кальция, чтобы вырастить крепкие кости.
В-третьих, слушай свое тело. Как ты себя чувствуешь, когда наелся? Когда вы едите, обращайте внимание на то, как чувствует себя ваше тело и когда ваш желудок чувствует себя комфортно насыщенным. Слишком много еды не сделает тебя комфортным и толстым.
В-четвертых, ограничить время экрана. Время скриншота - это время, когда вы смотрите телевизор, DVD и видео или используете компьютеры. Приятно делать больше упражнени

In [40]:
print(label)

  Как быть активным
  Как сделать себя важным
  Как сделать твоих родителей здоровыми


In [41]:
outp = model_predict(inp)

In [42]:
print(outp)

Привет, дорогие мальчики и девочки! Ты знаешь, как быть здоровым ребенком? Вот некоторые правила, которым ты должен следовать.
Во - первых, есть разные продукты, особенно фрукты и овощи. У вас может быть любимая еда, но вам лучше есть что-то другое, если вы едите разные продукты, вы, вероятно, получите больше питательных веществ, в которых нуждается ваше тело.
Во-вторых, пить воду и молоко как можно чаще. Когда ты действительно хочешь пить, холодная вода - это "Нет". 1 выбор. Молоко - это отличный напиток, который может дать вам больше кальция, чтобы вырастить крепкие кости.
В-третьих, слушай свое тело. Как ты себя чувствуешь, когда наелся? Когда вы едите, обращайте внимание на то, как чувствует себя ваше тело и когда ваш желудок чувствует себя комфортно насыщенным. Слишком много еды не сделает тебя комфортным и толстым.
В-четвертых, ограничить время экрана. Время скриншота - это время, когда вы смотрите телевизор, DVD и видео или используете компьютеры. Приятно делать больше упражнени